![Nuclio logo](https://nuclio.school/wp-content/uploads/2018/12/nucleoDS-newBlack.png)

# 1. Objetivos

*Saber que productos se están vendiendo más y qué productos tienen más éxito en cada tienda y ciudad.*

El presente notbook consiste en la creación de la tabla de ventas para responder el Objetivo N° 1 del proyecto de DS Market mediante la posterior visualización de los datos en PowerBI.

Output esperado:

![Descripción de la imagen](table.jpg)

# 2. Librerias

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 3. Cargar Datos

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_sales = pd.read_csv('/content/drive/MyDrive/COMPARTIDO_DSMARKET/Base_datos/item_sales.csv')
df_prices = pd.read_csv('/content/drive/MyDrive/COMPARTIDO_DSMARKET/Base_datos/item_prices.csv')
df_calendar = pd.read_csv('/content/drive/MyDrive/COMPARTIDO_DSMARKET/Base_datos/daily_calendar_with_events.csv')

In [ ]:
print('Rows: ', df_calendar.shape[0], '  Columns: ', df_calendar.shape[1])
df_calendar.head(2)

Rows:  1913   Columns:  5


,date,weekday,weekday_int,d,event
0,2011-01-29,Saturday,1,d_1,NaN
1,2011-01-30,Sunday,2,d_2,NaN


In [ ]:
print('Rows: ', df_prices.shape[0], '  Columns: ', df_prices.shape[1])
df_prices.head(2)

Rows:  6965706   Columns:  5


,item,category,store_code,yearweek,sell_price
0,ACCESORIES_1_001,ACCESORIES,NYC_1,201328.0,12.7414
1,ACCESORIES_1_001,ACCESORIES,NYC_1,201329.0,12.7414


In [ ]:
print('Rows: ', df_sales.shape[0], '  Columns: ', df_sales.shape[1])
df_sales.head(2)

Rows:  30490   Columns:  1920


,id,item,category,department,store,store_code,region,d_1,d_2,d_3,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,0,0,0,0,0,1,0,0,0,0


# 4. Preprocessing Tablas Individuales

### 4.1 Tabla Calendar

In [ ]:
# Intervalo de tiempo
df_calendar['date'].min(), df_calendar['date'].max()

('2011-01-29', '2016-04-24')

In [ ]:
# Creación del festivo 'Christmas'
christmas = ['2011-12-25','2012-12-25','2013-12-25','2014-12-25','2015-12-25']
filtro_christmas = df_calendar['date'].isin(christmas)

df_calendar.loc[filtro_christmas, 'event'] = 'Christmas'
df_calendar['event'].value_counts()

event
SuperBowl         6
Ramadan starts    5
Thanksgiving      5
Christmas         5
NewYear           5
Easter            5
Name: count, dtype: int64

In [ ]:
# Transformar a formato datetime la columna date y crear variables temporales
df_calendar['date'] = pd.to_datetime(df_calendar['date'])
df_calendar['yearweek'] = df_calendar['date'].dt.strftime('%G-%V')
df_calendar['month'] = df_calendar['date'].dt.month
df_calendar['week'] = df_calendar['date'].dt.isocalendar().week
df_calendar['yearmonth'] = df_calendar['date'].dt.strftime('%Y-%m')

df_calendar.head(2)

,date,weekday,weekday_int,d,event,yearweek,month,week,yearmonth
0,2011-01-29,Saturday,1,d_1,NaN,2011-04,1,4,2011-01
1,2011-01-30,Sunday,2,d_2,NaN,2011-04,1,4,2011-01


In [ ]:
# Checkear nulos y duplicados
print(f'Duplicados: {df_calendar.duplicated().sum()}')
print(f'Nulos: {df_calendar.isnull().sum().sum()}')

Duplicados: 0
Nulos: 1882


### 4.2 Tabla Sales

In [ ]:
# Agrupar por 'yearweek' y obtener listas de valores de 'd' para cada yearweek
agrupado = df_calendar.groupby('yearweek')['d'].apply(list).reset_index()
diccionario_resultado = dict(zip(agrupado['yearweek'], agrupado['d']))

In [ ]:
# Crear una lista de DataFrames que contienen las columnas sumadas
summed_columns = [df_sales[columnas].sum(axis=1) for columnas in diccionario_resultado.values()]

# Concatenar los DataFrames de la lista
df_summed = pd.concat(summed_columns, axis=1)

# Asignar los nombres de las columnas
df_summed.columns = diccionario_resultado.keys()

# Eliminar las columnas originales
columnas_a_eliminar = [col for lista_columnas in diccionario_resultado.values() for col in lista_columnas]
df_sales.drop(columns=columnas_a_eliminar, inplace=True)

# Concatenar el DataFrame original con el DataFrame de columnas sumadas
df_sales = pd.concat([df_sales, df_summed], axis=1)

df_sales.head(2)

,id,item,category,department,store,store_code,region,2011-04,2011-05,2011-06,...,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2016-13,2016-14,2016-15,2016-16
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,4,12,5,5,10,8,2,12,5,8
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,2,1,2,3,0,4,1,0,0,1


In [ ]:
# Seleccionar columnas a transponer
columnas_a_transponer = df_sales.columns[7:]
columnas_a_transponer

Index(['2011-04', '2011-05', '2011-06', '2011-07', '2011-08', '2011-09',
       '2011-10', '2011-11', '2011-12', '2011-13',
       ...
       '2016-07', '2016-08', '2016-09', '2016-10', '2016-11', '2016-12',
       '2016-13', '2016-14', '2016-15', '2016-16'],
      dtype='object', length=274)

In [ ]:
# Utilizar melt para convertir las columnas seleccionadas en filas
df_sales_melted = pd.melt(df_sales, id_vars=['id', 'item','category','department','store','store_code','region'], value_vars=columnas_a_transponer, var_name='yearweek', value_name='sales_quantity')
print('Rows: ', df_sales_melted.shape[0], '  Columns: ', df_sales_melted.shape[1])
print(df_sales_melted.info())
df_sales_melted.head(2)

# Checkear nulos y duplicados
print(f'Duplicados: {df_sales_melted.duplicated().sum()}')
print(f'Nulos: {df_sales_melted.isnull().sum().sum()}')

Rows:  8354260   Columns:  9
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8354260 entries, 0 to 8354259
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   id              object
 1   item            object
 2   category        object
 3   department      object
 4   store           object
 5   store_code      object
 6   region          object
 7   yearweek        object
 8   sales_quantity  int64 
dtypes: int64(1), object(8)
memory usage: 573.6+ MB
None
Duplicados: 0
Nulos: 0


### 4.3 Tabla Prices

In [ ]:
print('Rows: ', df_prices.shape[0], '  Columns: ', df_prices.shape[1])
df_prices.head(2)

Rows:  6965706   Columns:  5


,item,category,store_code,yearweek,sell_price
0,ACCESORIES_1_001,ACCESORIES,NYC_1,201328.0,12.7414
1,ACCESORIES_1_001,ACCESORIES,NYC_1,201329.0,12.7414


In [ ]:
# Checkear nulos y duplicados
print(f'Duplicados: {df_prices.duplicated().sum()}')
print(f'Nulos: {df_prices.isnull().sum().sum()}')

Duplicados: 212120
Nulos: 243920


In [ ]:
# Eliminar duplicados y nulos
df_prices = df_prices.drop_duplicates()
df_prices = df_prices.dropna()
print(f'Nulos: {df_prices.isnull().sum().sum()}')
print('Rows: ', df_prices.shape[0], '  Columns: ', df_prices.shape[1])
print(f'Duplicados: {df_prices.duplicated().sum()}')
df_prices.head(2)

Nulos: 0
Rows:  6721786   Columns:  5
Duplicados: 0


,item,category,store_code,yearweek,sell_price
0,ACCESORIES_1_001,ACCESORIES,NYC_1,201328.0,12.7414
1,ACCESORIES_1_001,ACCESORIES,NYC_1,201329.0,12.7414


In [ ]:
# Generar las variables temporales para que coincidan con las otras tablas
df_prices['yearweek'] = df_prices['yearweek'].astype(str)
df_prices['yearweek'] = df_prices['yearweek'].str[:6]
df_prices['year'] = df_prices['yearweek'].str[:4]
df_prices['week'] = df_prices['yearweek'].str[4:]
df_prices['yearweek'] = df_prices['yearweek'].str.slice(0, 4) + '-' + df_prices['yearweek'].str.slice(4)
df_prices.head(2)

,item,category,store_code,yearweek,sell_price,year,week
0,ACCESORIES_1_001,ACCESORIES,NYC_1,2013-28,12.7414,2013,28
1,ACCESORIES_1_001,ACCESORIES,NYC_1,2013-29,12.7414,2013,29


In [ ]:
# Crear la columna 'item_store_code' que será igual a la columna'id' de la tabla sales
df_prices['item_store_code'] = df_prices['item'].str.cat(df_prices['store_code'], sep='_')
df_prices.head(2)

,item,category,store_code,yearweek,sell_price,year,week,item_store_code
0,ACCESORIES_1_001,ACCESORIES,NYC_1,2013-28,12.7414,2013,28,ACCESORIES_1_001_NYC_1
1,ACCESORIES_1_001,ACCESORIES,NYC_1,2013-29,12.7414,2013,29,ACCESORIES_1_001_NYC_1


In [ ]:
# Calculo el promedio de sell_price
df_prices['mean_sell_price'] = df_prices.groupby(['item_store_code'])['sell_price'].transform(lambda x: x.mean())

In [ ]:
# Checkear nuevamente los valores nulos
print(f'Nulos: {df_prices.isnull().sum().sum()}')
print(f'Duplicados: {df_prices.duplicated().sum()}')
print('Rows: ', df_prices.shape[0], '  Columns: ', df_prices.shape[1])
df_prices.head(2)

Nulos: 0
Duplicados: 0
Rows:  6721786   Columns:  9


,item,category,store_code,yearweek,sell_price,year,week,item_store_code,mean_sell_price
0,ACCESORIES_1_001,ACCESORIES,NYC_1,2013-28,12.7414,2013,28,ACCESORIES_1_001_NYC_1,11.012579
1,ACCESORIES_1_001,ACCESORIES,NYC_1,2013-29,12.7414,2013,29,ACCESORIES_1_001_NYC_1,11.012579


In [ ]:
df_prices_1 = df_prices.groupby('item_store_code')['mean_sell_price'].mean().reset_index()
print(f'Nulos: {df_prices_1.isnull().sum().sum()}')
print(f'Duplicados: {df_prices_1.duplicated().sum()}')
print('Rows: ', df_prices_1.shape[0], '  Columns: ', df_prices_1.shape[1])
df_prices_1.head(2)

Nulos: 0
Duplicados: 0
Rows:  30490   Columns:  2


,item_store_code,mean_sell_price
0,ACCESORIES_1_001_BOS_1,10.968483
1,ACCESORIES_1_001_BOS_2,10.997743


# 5. Union de las Tablas

In [ ]:
# Unir la columna sell_price de la tabla prices a la tabla df_sales_melted
sell_price_diccionario = dict(zip(df_prices_1['item_store_code'], df_prices_1['mean_sell_price']))
df_sales_melted['mean_sell_price'] = df_sales_melted['id'].map(sell_price_diccionario)
print(f'Duplicados: {df_sales_melted.duplicated().sum()}')
print(f'Nulos: {df_sales_melted.isnull().sum().sum()}')
df_sales_melted.head(2)

Duplicados: 0
Nulos: 0


,id,item,category,department,store,store_code,region,yearweek,sales_quantity,mean_sell_price
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,2011-04,0,11.012579
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,2011-04,0,5.280100


In [ ]:
# De la tabla calendar agrego el 'yearmonth' a la tabla df_sales_melted
df_calendar_1 = df_calendar.groupby('yearweek')['yearmonth'].max().reset_index()
df_calendar_1.head(2)

,yearweek,yearmonth
0,2011-04,2011-01
1,2011-05,2011-02


In [ ]:
df_sales_melted = pd.merge(df_sales_melted, df_calendar_1[['yearweek', 'yearmonth']], on='yearweek', how='left')
print(f'Duplicados: {df_sales_melted.duplicated().sum()}')
print(f'Nulos: {df_sales_melted.isnull().sum().sum()}')
df_sales_melted.head(2)

Duplicados: 0
Nulos: 0


,id,item,category,department,store,store_code,region,yearweek,sales_quantity,mean_sell_price,yearmonth
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,2011-04,0,11.012579,2011-01
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,2011-04,0,5.280100,2011-01


In [ ]:
# Crear la columna month y year
df_sales_melted['year'] = df_sales_melted['yearmonth'].str[:4]
df_sales_melted['month'] = df_sales_melted['yearmonth'].str[-2:]
print(f'Duplicados: {df_sales_melted.duplicated().sum()}')
print(f'Nulos: {df_sales_melted.isnull().sum().sum()}')
print('Rows: ', df_sales_melted.shape[0], '  Columns: ', df_sales_melted.shape[1])
df_sales_melted.head(2)

Duplicados: 0
Nulos: 0
Rows:  8354260   Columns:  13


,id,item,category,department,store,store_code,region,yearweek,sales_quantity,mean_sell_price,yearmonth,year,month
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,2011-04,0,11.012579,2011-01,2011,01
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,2011-04,0,5.280100,2011-01,2011,01


In [ ]:
df_sales_pivot = df_sales_melted.copy()

In [ ]:
del(df_sales_pivot['yearweek'])
print(f'Duplicados: {df_sales_pivot.duplicated().sum()}')
print(f'Nulos: {df_sales_pivot.isnull().sum().sum()}')
print('Rows: ', df_sales_pivot.shape[0], '  Columns: ', df_sales_pivot.shape[1])
df_sales_pivot.head(2)

Duplicados: 3117419
Nulos: 0
Rows:  8354260   Columns:  12


,id,item,category,department,store,store_code,region,sales_quantity,mean_sell_price,yearmonth,year,month
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,11.012579,2011-01,2011,01
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,5.280100,2011-01,2011,01


In [ ]:
df_sales_pivot['sales_quantity'] = df_sales_pivot.groupby(['yearmonth', 'id'])['sales_quantity'].transform('sum')
df_sales_pivot['mean_sell_price'] = df_sales_pivot.groupby(['yearmonth', 'id'])['mean_sell_price'].transform('mean')
print(f'Duplicados: {df_sales_pivot.duplicated().sum()}')
print(f'Nulos: {df_sales_pivot.isnull().sum().sum()}')
print('Rows: ', df_sales_pivot.shape[0], '  Columns: ', df_sales_pivot.shape[1])
print(len(df_sales_pivot.groupby(['yearmonth', 'id'])))
df_sales_pivot.head(2)

Duplicados: 6402900
Nulos: 0
Rows:  8354260   Columns:  12
1951360


,id,item,category,department,store,store_code,region,sales_quantity,mean_sell_price,yearmonth,year,month
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,11.012579,2011-01,2011,01
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,5.280100,2011-01,2011,01


In [ ]:
# Eliminar duplicados para que me quede la tabla por id y month_Sales
df_sales_pivot = df_sales_pivot.drop_duplicates()
print(f'Duplicados: {df_sales_pivot.duplicated().sum()}')
print(f'Nulos: {df_sales_pivot.isnull().sum().sum()}')
print('Rows: ', df_sales_pivot.shape[0], '  Columns: ', df_sales_pivot.shape[1])
print(len(df_sales_pivot.groupby(['yearmonth', 'id'])))
df_sales_pivot.head(2)

Duplicados: 0
Nulos: 0
Rows:  1951360   Columns:  12
1951360


,id,item,category,department,store,store_code,region,sales_quantity,mean_sell_price,yearmonth,year,month
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,11.012579,2011-01,2011,01
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,5.280100,2011-01,2011,01


In [ ]:
# Crear la columna revenue
df_sales_pivot['revenue'] = df_sales_pivot['sales_quantity'] * df_sales_pivot['mean_sell_price']
print(f'Duplicados: {df_sales_pivot.duplicated().sum()}')
print(f'Nulos: {df_sales_pivot.isnull().sum().sum()}')
print('Rows: ', df_sales_pivot.shape[0], '  Columns: ', df_sales_pivot.shape[1])
df_sales_pivot.head(2)

Duplicados: 0
Nulos: 0
Rows:  1951360   Columns:  13


,id,item,category,department,store,store_code,region,sales_quantity,mean_sell_price,yearmonth,year,month,revenue
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,11.012579,2011-01,2011,01,0.0
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,5.280100,2011-01,2011,01,0.0


# 6. Tabla Final

In [ ]:
df_sales_final = df_sales_pivot.copy()

In [ ]:
# Ordenar la tabla
nuevo_orden_columnas = ['id', 'item', 'region', 'store', 'store_code', 'department','category', 'month', 'year', 'sales_quantity', 'mean_sell_price', 'revenue']
df_sales_final = df_sales_final[nuevo_orden_columnas].copy()
df_sales_final.rename(columns=lambda x: x.upper(), inplace=True)
df_sales_final.head(2)

,ID,ITEM,REGION,STORE,STORE_CODE,DEPARTMENT,CATEGORY,MONTH,YEAR,SALES_QUANTITY,MEAN_SELL_PRICE,REVENUE
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,New York,Greenwich_Village,NYC_1,ACCESORIES_1,ACCESORIES,01,2011,0,11.012579,0.0
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,New York,Greenwich_Village,NYC_1,ACCESORIES_1,ACCESORIES,01,2011,0,5.280100,0.0


In [ ]:
df_sales_final['STORE_CODE'] = df_sales_final['STORE_CODE'].str.replace('_', ' ')
df_sales_final['DEPARTMENT'] = df_sales_final['DEPARTMENT'].str.replace('_', ' ')
df_sales_final['DATE'] = df_sales_final['MONTH'] + str('-') + df_sales_final['YEAR']

# Necesitamos conservar la columna inicial para juntar tablas en PowerBI
df_sales_final['ITEM_2'] = df_sales_final['ITEM'].str.replace('ACCESORIES', 'ACC')
df_sales_final['ITEM_2'] = df_sales_final['ITEM'].str.replace('SUPERMARKET', 'SUP')
df_sales_final['ITEM_2'] = df_sales_final['ITEM'].str.replace('HOME_&_GARDEN', 'H&G')

df_sales_final['DEPARTMENT'] = df_sales_final['DEPARTMENT'].str.replace('ACCESORIES_', 'Accesories ')
df_sales_final['DEPARTMENT'] = df_sales_final['DEPARTMENT'].str.replace('SUPERMARKET_', 'Supermarket ')
df_sales_final['DEPARTMENT'] = df_sales_final['DEPARTMENT'].str.replace('HOME_&_GARDEN_', 'Home & Garden ')

df_sales_final['CATEGORY'] = df_sales_final['CATEGORY'].str.replace('ACCESORIES', 'Accesories')
df_sales_final['CATEGORY'] = df_sales_final['CATEGORY'].str.replace('SUPERMARKET', 'Supermarket')
df_sales_final['CATEGORY'] = df_sales_final['CATEGORY'].str.replace('HOME_&_GARDEN', 'Home & Garden')
df_sales_final['STORE'] = df_sales_final['STORE'].str.replace('_', ' ')

df_sales_final = df_sales_final.rename(columns={'MEAN_SELL_PRICE': 'SELL_PRICE'})

# 7. Tabla Check

In [ ]:
# Algunos puntos de control:
df_sales_final['STORE_CODE'].value_counts()

STORE_CODE
NYC 1    195136
NYC 2    195136
NYC 3    195136
NYC 4    195136
BOS 1    195136
BOS 2    195136
BOS 3    195136
PHI 1    195136
PHI 2    195136
PHI 3    195136
Name: count, dtype: int64

In [ ]:
df_sales_final['CATEGORY'].value_counts()

CATEGORY
Supermarket      919680
Home & Garden    670080
Accesories       361600
Name: count, dtype: int64

In [ ]:
df_sales_final['REVENUE'].sort_values()

0              0.000000
2807856        0.000000
2807857        0.000000
2807860        0.000000
2807861        0.000000
               ...     
6513009    24844.667900
7580159    25337.862300
7854569    25738.582750
7173592    26617.337204
6787419    32304.233200
Name: REVENUE, Length: 1951360, dtype: float64

In [ ]:
print(df_sales_final['SELL_PRICE'].min())
print(df_sales_final['SELL_PRICE'].max())

0.22800000000000004
40.607204455445554


In [ ]:
print(df_sales_final['SALES_QUANTITY'].min())
print(df_sales_final['SALES_QUANTITY'].max())

0
13188


In [ ]:
print(df_sales_final['REVENUE'].min())
print(df_sales_final['REVENUE'].max())

0.0
32304.2332


# 8. Guardar Tabla

In [ ]:
# Convertir la columna 'DATE' al formato de fecha
df_sales_final['DATE'] = pd.to_datetime(df_sales_final['DATE'], format='%m-%Y')

In [ ]:
df_sales_final.head()

,ID,ITEM,REGION,STORE,STORE_CODE,DEPARTMENT,CATEGORY,MONTH,YEAR,SALES_QUANTITY,SELL_PRICE,REVENUE,DATE,ITEM_2
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,New York,Greenwich Village,NYC 1,ACCESORIES 1,Accesories,01,2011,0,11.012579,0.0,2011-01-01,ACCESORIES_1_001
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,New York,Greenwich Village,NYC 1,ACCESORIES 1,Accesories,01,2011,0,5.280100,0.0,2011-01-01,ACCESORIES_1_002
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,New York,Greenwich Village,NYC 1,ACCESORIES 1,Accesories,01,2011,0,3.950100,0.0,2011-01-01,ACCESORIES_1_003
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,New York,Greenwich Village,NYC 1,ACCESORIES 1,Accesories,01,2011,0,6.018299,0.0,2011-01-01,ACCESORIES_1_004
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,New York,Greenwich Village,NYC 1,ACCESORIES 1,Accesories,01,2011,0,3.912324,0.0,2011-01-01,ACCESORIES_1_005


In [ ]:
# Guardar en CSV
df_sales_final.to_csv('/content/drive/MyDrive/COMPARTIDO_DSMARKET/1. Visualizacion/PowerBi/SALES_DSMARKET.csv', index=False)